In [ ]:
import sys
sys.path.append('../../')
from fig1_calibration import create_sim as cs
bounds = cs.define_pars(which='bounds')
bounds['tn'][0] = 5.0
keys = list(bounds.keys())
import pandas as pd
from scipy.stats import qmc

diagnoses = pd.read_csv('../posterior/calibrated-diagnoses.csv')
death = pd.read_csv('../posterior/calibrated-death.csv')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(ncols=2,nrows=2)
ax = ax.flatten()
overlap = pd.merge(diagnoses.round(2).reset_index(),death.round(2).reset_index(),on=keys)
joint_post = diagnoses.reset_index()
joint_post = joint_post.loc[joint_post['index'].isin(overlap['index_x'].values)]

In [ ]:
posterior = pd.DataFrame(qmc.scale(joint_post[keys].values,
                 l_bounds = [bounds[k][0] for k in keys],
                 u_bounds = [bounds[k][1] for k in keys]),
                 columns=keys)
posterior

In [ ]:
import numpy as np
max_rand_seed = pd.read_csv('../data/sims_combined-wave004.csv')['rand_seed'].max()
n = 50
new_samples = posterior.sample(n=n,random_state=1)

new_samples['rand_seed'] = max_rand_seed + 1
new_samples['rand_seed'] += np.arange(n)
new_samples.to_csv('posterior-samples.csv',index=False)